In [ ]:
# Setup
import sys
from pathlib import Path

notebook_dir = Path.cwd()
project_root = notebook_dir.parent if notebook_dir.name == 'notebooks' else notebook_dir
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from core import (
    MonteCarloEngine,
    PolicyScenario,
    EconomicParameters,
    SensitivityAnalyzer,
    get_policy_by_type,
    PolicyType,
    simulate_healthcare_years,
)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# Set random seed for reproducibility
np.random.seed(42)

print("✅ Setup complete!")

---

## 1. The Problem with Point Estimates

### CBO's Approach
The Congressional Budget Office publishes **single numbers**:

```
2034 Deficit: $2.3 trillion
2034 Debt-to-GDP: 122%
```

### The Problem
Reality is **uncertain**. Economic variables fluctuate:
- GDP growth could be 1% or 4%
- Inflation could be 2% or 5%
- Interest rates change with Fed policy

### What We Need
**Probability distributions**, not point estimates:

```
2034 Deficit: $2.3T [90% CI: $1.8T - $3.1T]
```

In [ ]:
# Demonstrate point estimate vs distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Point estimate (CBO-style)
ax1 = axes[0]
years = range(2024, 2035)
point_estimate = [1.7, 1.8, 1.9, 2.0, 2.1, 2.1, 2.2, 2.2, 2.3, 2.3, 2.3]
ax1.plot(years, point_estimate, 'b-', linewidth=3, marker='o')
ax1.set_xlabel('Year')
ax1.set_ylabel('Deficit ($ Trillions)')
ax1.set_title('Traditional: Single Point Estimate', fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 4)

# Monte Carlo distribution
ax2 = axes[1]
# Simulate uncertainty growing over time
mean_path = np.array(point_estimate)
std_growth = np.linspace(0.1, 0.4, len(years))  # Uncertainty grows with time
p5 = mean_path - 1.645 * std_growth * mean_path
p95 = mean_path + 1.645 * std_growth * mean_path

ax2.plot(years, mean_path, 'b-', linewidth=2, label='Mean')
ax2.fill_between(years, p5, p95, alpha=0.3, color='blue', label='90% Confidence Interval')
ax2.set_xlabel('Year')
ax2.set_ylabel('Deficit ($ Trillions)')
ax2.set_title('Monte Carlo: Probability Distribution', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 4)

plt.tight_layout()
plt.show()

print("💡 Key Insight: Uncertainty grows over time. The further out we project, the wider the range.")

---

## 2. Monte Carlo Basics

### The Method

1. **Define probability distributions** for uncertain parameters
2. **Randomly sample** from those distributions
3. **Run the model** with sampled values
4. **Repeat** thousands of times
5. **Analyze** the distribution of outcomes

### Example: Coin Flip

If we flip a coin 10 times, how many heads?
- **Point estimate**: 5 heads (expected value)
- **Monte Carlo**: Simulate 10,000 trials, see the full distribution

In [ ]:
# Monte Carlo demonstration: Coin flips
n_flips = 10
n_simulations = 10000

# Simulate
results = np.random.binomial(n_flips, 0.5, n_simulations)

# Visualize
fig, ax = plt.subplots(figsize=(10, 5))

# Histogram
counts, bins, _ = ax.hist(results, bins=range(12), density=True, alpha=0.7, color='steelblue', edgecolor='black')

# Add theoretical distribution
x = np.arange(0, 11)
theoretical = stats.binom.pmf(x, n_flips, 0.5)
ax.plot(x, theoretical, 'ro-', linewidth=2, markersize=8, label='Theoretical')

ax.axvline(x=5, color='green', linestyle='--', linewidth=2, label='Point Estimate (5)')
ax.set_xlabel('Number of Heads')
ax.set_ylabel('Probability')
ax.set_title(f'Monte Carlo Simulation: {n_simulations:,} Trials of {n_flips} Coin Flips', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"📊 Results from {n_simulations:,} simulations:")
print(f"   • Mean: {results.mean():.2f} heads")
print(f"   • Std Dev: {results.std():.2f}")
print(f"   • 90% CI: [{np.percentile(results, 5):.0f}, {np.percentile(results, 95):.0f}] heads")

---

## 3. Convergence and Sample Size

How many iterations do we need?

**Law of Large Numbers**: As sample size increases, the sample mean converges to the true mean.

Let's see how estimates stabilize as we increase iterations:

In [ ]:
# Demonstrate convergence
max_iterations = 10000
all_samples = np.random.normal(2.5, 0.5, max_iterations)  # GDP growth example

# Calculate running mean
running_means = np.cumsum(all_samples) / np.arange(1, max_iterations + 1)

# Plot
fig, ax = plt.subplots(figsize=(12, 5))

ax.plot(running_means, 'b-', linewidth=1, alpha=0.7)
ax.axhline(y=2.5, color='red', linestyle='--', linewidth=2, label='True Mean (2.5%)')
ax.axhline(y=2.5 + 0.01, color='green', linestyle=':', alpha=0.5)
ax.axhline(y=2.5 - 0.01, color='green', linestyle=':', alpha=0.5, label='±0.01% tolerance')

ax.set_xlabel('Number of Iterations')
ax.set_ylabel('Running Mean GDP Growth (%)')
ax.set_title('Monte Carlo Convergence: Mean Estimate Stabilizes with More Iterations', fontweight='bold')
ax.set_xlim(0, max_iterations)
ax.set_ylim(2.3, 2.7)
ax.legend()
ax.grid(True, alpha=0.3)

# Add annotations
ax.annotate('High variance\n(few samples)', xy=(100, running_means[99]), fontsize=10)
ax.annotate('Converged\n(stable estimate)', xy=(8000, running_means[7999]), fontsize=10)

plt.tight_layout()
plt.show()

print("💡 PoliSim uses 100,000+ iterations for production-quality uncertainty estimates.")

---

## 4. PoliSim's Stochastic Parameters

PoliSim models uncertainty in these key parameters:

| Parameter | Distribution | Mean | Std Dev | Source |
|-----------|-------------|------|---------|--------|
| GDP Growth | Normal | 2.2% | 0.8% | Historical |
| Inflation | Normal | 2.5% | 0.5% | Fed target |
| Interest Rate | AR(1) | 4.5% | 1.0% | Treasury |
| Healthcare Inflation | Lognormal | 3.5% | 1.2% | CMS |
| Unemployment | Normal | 4.5% | 1.5% | BLS |

In [ ]:
# Visualize parameter distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# GDP Growth
ax1 = axes[0, 0]
gdp_samples = np.random.normal(2.2, 0.8, 10000)
ax1.hist(gdp_samples, bins=50, density=True, alpha=0.7, color='steelblue')
ax1.axvline(x=2.2, color='red', linestyle='--', label='Mean')
ax1.set_xlabel('GDP Growth (%)')
ax1.set_title('GDP Growth Distribution')
ax1.legend()

# Inflation
ax2 = axes[0, 1]
inflation_samples = np.random.normal(2.5, 0.5, 10000)
ax2.hist(inflation_samples, bins=50, density=True, alpha=0.7, color='green')
ax2.axvline(x=2.5, color='red', linestyle='--', label='Mean')
ax2.set_xlabel('Inflation (%)')
ax2.set_title('Inflation Distribution')
ax2.legend()

# Interest Rate
ax3 = axes[0, 2]
interest_samples = np.random.normal(4.5, 1.0, 10000)
ax3.hist(interest_samples, bins=50, density=True, alpha=0.7, color='orange')
ax3.axvline(x=4.5, color='red', linestyle='--', label='Mean')
ax3.set_xlabel('Interest Rate (%)')
ax3.set_title('Interest Rate Distribution')
ax3.legend()

# Healthcare Inflation (Lognormal)
ax4 = axes[1, 0]
healthcare_samples = np.random.lognormal(np.log(3.5), 0.3, 10000)
ax4.hist(healthcare_samples, bins=50, density=True, alpha=0.7, color='purple')
ax4.axvline(x=3.5, color='red', linestyle='--', label='Mode')
ax4.set_xlabel('Healthcare Inflation (%)')
ax4.set_title('Healthcare Inflation (Lognormal)')
ax4.legend()

# Unemployment
ax5 = axes[1, 1]
unemp_samples = np.random.normal(4.5, 1.5, 10000)
ax5.hist(unemp_samples, bins=50, density=True, alpha=0.7, color='brown')
ax5.axvline(x=4.5, color='red', linestyle='--', label='Mean')
ax5.set_xlabel('Unemployment (%)')
ax5.set_title('Unemployment Distribution')
ax5.legend()

# Combined fiscal outcome
ax6 = axes[1, 2]
ax6.text(0.5, 0.5, 'Combined outcomes\nare shown in\nfiscal projections', 
         ha='center', va='center', fontsize=12, transform=ax6.transAxes)
ax6.set_title('Combined Fiscal Impact')
ax6.axis('off')

plt.tight_layout()
plt.show()

---

## 5. Running Monte Carlo in PoliSim

Let's run actual Monte Carlo simulations and analyze the results.

In [ ]:
# Run Monte Carlo simulation with PoliSim
print("🔄 Running Monte Carlo simulation (comparing iteration counts)...\n")

policy = get_policy_by_type(PolicyType.USGHA)
iteration_counts = [100, 1000, 5000]
results_by_iterations = {}

for n_iter in iteration_counts:
    # Note: simulate_healthcare_years is deterministic, but we can demonstrate the concept
    # by adding noise to parameters
    base_gdp = 29e12
    gdp_samples = np.random.normal(base_gdp, base_gdp * 0.02, n_iter)
    
    final_spending = []
    for gdp in gdp_samples:
        result = simulate_healthcare_years(
            policy=policy,
            base_gdp=gdp,
            initial_debt=35e12,
            years=10,
            population=335e6,
            gdp_growth=np.random.normal(0.025, 0.005),
            start_year=2025
        )
        final_spending.append(result.iloc[-1]['Health Spending ($)'])
    
    results_by_iterations[n_iter] = np.array(final_spending)
    
    mean = np.mean(final_spending) / 1e12
    std = np.std(final_spending) / 1e12
    print(f"{n_iter:,} iterations: Mean=${mean:.2f}T, Std=${std:.3f}T")

print("\n✅ Simulations complete!")

In [ ]:
# Visualize results by iteration count
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, (n_iter, samples) in enumerate(results_by_iterations.items()):
    ax = axes[i]
    samples_t = samples / 1e12
    
    ax.hist(samples_t, bins=30, density=True, alpha=0.7, color='steelblue', edgecolor='black')
    ax.axvline(x=np.mean(samples_t), color='red', linestyle='--', linewidth=2, label=f'Mean: ${np.mean(samples_t):.2f}T')
    ax.axvline(x=np.percentile(samples_t, 5), color='orange', linestyle=':', linewidth=2)
    ax.axvline(x=np.percentile(samples_t, 95), color='orange', linestyle=':', linewidth=2, label='90% CI')
    
    ax.set_xlabel('2034 Healthcare Spending ($ Trillions)')
    ax.set_ylabel('Density')
    ax.set_title(f'{n_iter:,} Iterations', fontweight='bold')
    ax.legend()

plt.tight_layout()
plt.show()

---

## 6. Interpreting Results

### Key Metrics

| Metric | Meaning |
|--------|--------|
| **Mean** | Average outcome across all simulations |
| **Median** | Middle value (50th percentile) |
| **Std Dev** | Spread of outcomes |
| **90% CI** | 5th to 95th percentile range |
| **5th percentile** | "Bad case" scenario |
| **95th percentile** | "Good case" scenario |

In [ ]:
# Comprehensive statistics from our simulation
samples = results_by_iterations[5000] / 1e12  # Use largest sample

print("📊 Monte Carlo Results: 2034 Healthcare Spending")
print("=" * 50)
print(f"\n📈 Central Tendency:")
print(f"   • Mean: ${np.mean(samples):.3f} Trillion")
print(f"   • Median: ${np.median(samples):.3f} Trillion")
print(f"   • Mode (approx): ${stats.mode(np.round(samples, 2), keepdims=True).mode[0]:.3f} Trillion")

print(f"\n📉 Dispersion:")
print(f"   • Std Dev: ${np.std(samples):.4f} Trillion")
print(f"   • IQR: ${np.percentile(samples, 75) - np.percentile(samples, 25):.4f} Trillion")

print(f"\n🎯 Confidence Intervals:")
print(f"   • 50% CI: [${np.percentile(samples, 25):.3f}T, ${np.percentile(samples, 75):.3f}T]")
print(f"   • 90% CI: [${np.percentile(samples, 5):.3f}T, ${np.percentile(samples, 95):.3f}T]")
print(f"   • 95% CI: [${np.percentile(samples, 2.5):.3f}T, ${np.percentile(samples, 97.5):.3f}T]")

print(f"\n⚠️ Tail Risks:")
print(f"   • 5th percentile (bad case): ${np.percentile(samples, 5):.3f} Trillion")
print(f"   • 1st percentile (extreme): ${np.percentile(samples, 1):.3f} Trillion")

---

## 🎓 What You've Learned

✅ Point estimates hide uncertainty - probability distributions are more informative  
✅ Monte Carlo simulation samples from parameter distributions thousands of times  
✅ Convergence requires sufficient iterations (PoliSim uses 100K+)  
✅ Confidence intervals show the range of likely outcomes  
✅ Tail risks help identify worst-case scenarios  

---

## 📚 Next Steps

| Notebook | Topic | Time |
|----------|-------|------|
| **06** | Tax Policy Modeling | 45-60 min |
| **07** | Policy Extraction | 30-45 min |
| **08** | API Integration | 30-45 min |

---

*Continue to Notebook 06: Tax Policy Modeling* →